# Import required libraries

In [1]:
import pandas as pd
from tqdm import tqdm
import datetime


#https://www.kaggle.com/thegurusteam/spanish-high-speed-rail-system-ticket-pricing

In [2]:
tqdm.pandas()

# Read dataframe

In [3]:
df=pd.read_csv('thegurus-opendata-renfe-trips.csv')

In [4]:
df.shape

(38753060, 14)

filter rows without price

In [5]:
df = df[df['price'].notna()]

In [6]:
df.shape

(29593111, 14)

sample data, if order to reduce the amount of data

In [17]:
df=df.sample(n=5000, replace=False, random_state=1)

In [18]:
df.shape

(5000, 20)

In [19]:
df.head()

,id,company,origin,destination,departure,arrival,duration,vehicle_type,vehicle_class,price,fare,seats,meta,insert_date,departureDate,departureDay,departureMonth,departureHour,departureWeekday,departureTime
14951893,14951894,renfe,SEVILLA,MADRID,2019-10-06 18:45:00,2019-10-06 21:17:00,2.53,AVE,None,76.30,None,NaN,{},2019-08-20 05:38:14,2019-10-06 18:45:00,6,10,18,6,lateNight
6724228,6724229,renfe,SEVILLA,MADRID,2019-05-23 15:45:00,2019-05-23 18:15:00,2.50,AVE,None,76.30,None,NaN,{},2019-05-19 01:25:20,2019-05-23 15:45:00,23,5,15,3,midday
15736018,15736019,renfe,BARCELONA,MADRID,2019-11-19 18:00:00,2019-11-19 21:10:00,3.17,AVE,None,102.15,None,NaN,{},2019-11-11 17:08:02,2019-11-19 18:00:00,19,11,18,1,lateNight
20343550,20343551,renfe,MADRID,HUESCA,2020-03-02 17:30:00,2020-03-02 20:07:00,2.62,AVE-MD,None,57.50,None,NaN,"{""Turista con enlace"": {""Flexible"": {""price"": ...",2020-02-22 23:55:31,2020-03-02 17:30:00,2,3,17,0,afternoon
1466610,1466611,renfe,MADRID,VALENCIA,2019-04-29 11:10:00,2019-04-29 13:25:00,2.25,INTERCITY,None,43.75,None,NaN,{},2019-04-17 08:20:00,2019-04-29 11:10:00,29,4,11,0,midmorning


# Feature engineering

Extrac: dat, day of the week, month, hour from departure time

In [10]:
def processDate(x):
    x['departureDate']=datetime.datetime.strptime(x["departure"], '%Y-%m-%d %H:%M:%S')
    x['departureDay']=x["departureDate"].day
    x['departureMonth']=x["departureDate"].month
    x['departureHour']=x["departureDate"].hour
    x['departureWeekday']=x["departureDate"].weekday()
    return x

df=df.progress_apply(processDate, axis=1)

#df['departureDate']=df.progress_apply(lambda x: datetime.datetime.strptime(x["departure"], '%Y-%m-%d %H:%M:%S'),axis=1)
#df['departureDay']=df.progress_apply(lambda x: x["departureDate"].day,axis=1)
#df['departureMonth']=df.progress_apply(lambda x: x["departureDate"].month,axis=1)
#df['departureHour']=df.progress_apply(lambda x: x["departureDate"].hour,axis=1)
#df['departureWeekday']=df.progress_apply(lambda x: x["departureDate"].weekday(),axis=1)

100%|██████████| 500000/500000 [34:08<00:00, 244.09it/s]


In [11]:
df.shape

(500000, 19)

recode departure time

In [12]:
df['departureTime']=""
df.loc[(df['departureHour']<6),'departureTime' ] = "earlyMorning"
df.loc[((df['departureHour']>=6)&(df['departureHour']<9)),'departureTime' ] = "morning"
df.loc[((df['departureHour']>=9)&(df['departureHour']<12)),'departureTime' ] = "midmorning"
df.loc[((df['departureHour']>=12)&(df['departureHour']<16)),'departureTime' ] = "midday"
df.loc[((df['departureHour']>=16)&(df['departureHour']<18)),'departureTime' ] = "afternoon"
df.loc[((df['departureHour']>=18)&(df['departureHour']<21)),'departureTime' ] = "lateNight"
df.loc[(df['departureHour']>=21),'departureTime' ] = "night"


# Vars 

In [13]:
categoricalVar=['origin', 'destination','vehicle_type','vehicle_class', 'fare','departureWeekday', 'departureTime']
target='price'
continueVar=['duration','departureDay','departureMonth']


# Filter unknown obs

In [14]:
df['vehicle_class']=df['vehicle_class'].fillna("unknow", inplace=True)
df['fare']=df['fare'].fillna("unknow", inplace=True)

# Create dummys variables

In [20]:
X=df[continueVar]
y=df[target]
print(X.shape)
for i in categoricalVar:
    X=pd.concat([X, pd.get_dummies(pd.Series(df[i]), drop_first=True)], axis=1)
print(X.shape)
print(y.shape)


(5000, 3)
(5000, 100)
(5000,)


# Save clean data

In [21]:
data=X.copy()
data['y']=y

data.to_csv('dataLite.csv',sep=";",index=False)

# Model

## Split data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

## Train

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(max_depth=200, random_state=11)
clf.fit(X_train, y_train)

## Predict

In [ ]:
yhat_train=clf.predict(X_train)
yhat_valid=clf.predict(X_valid)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import explained_variance_score

## Evaluate

In [ ]:
print(f"MAE train: {mean_absolute_error(y_train, yhat_train)}")
print(f"MAE valid: {mean_absolute_error(y_valid, yhat_valid)}")
print(f"MSE train: {mean_squared_error(y_train, yhat_train)}")
print(f"MSE valid: {mean_squared_error(y_valid, yhat_valid)}")
print(f"RMSE train: {sqrt(mean_squared_error(y_train, yhat_train))}")
print(f"RMSE valid: {sqrt(mean_squared_error(y_valid, yhat_valid))}")
print(f"explained_variance train: {explained_variance_score(y_train, yhat_train, multioutput='uniform_average')}")
print(f"explained_variance valid: {explained_variance_score(y_valid, yhat_valid, multioutput='uniform_average')}")


